In [2]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import pickle
import os

In [0]:
path = dataiku.Folder("XXZ13n5V")
path=path.get_path()
base = os.path.join(path, "base_html.csv")
base

In [3]:
path = dataiku.Folder("XXZ13n5V")
path=path.get_path()
base = os.path.join(path, "base_html.csv")
thesaurus=os.path.join(path,"Thesaurus0.pickle")
projets=os.path.join(path,"num_theme.csv")

df_html=pd.read_csv(base)
df_html

thesaurus=pickle.load(open(thesaurus,'rb')) #dictionnaire de W2V
projets=pd.read_csv(projets)

In [4]:
thesaurus

,Enjeux,Dictionnaire
0,Biodiversité,"[biodiversité, milieu naturel, habitat naturel..."
1,Paysage et qualité de vie,"[paysage, cadre de vie, qualité de vie, intégr..."
2,Santé et sécurité des personnes,"[santé humaine, toxicité, bactériologique, qua..."
3,"Effets globaux (climat, énergie, ressources...)","[climat, changement climatique, réchauffement,..."
4,Préservation des sols,"[sols, artificialisation, consommation fonciér..."
5,Qualité de leau et ressource,"[eaux usées, eaux pluviales, eau de pluie, hyd..."
6,Déplacements,"[déplacements, transport, mobilité, véhicules,..."
7,Gestion des déchets,"[déchets, épandage, compostage, recyclage, mét..."


In [5]:
import re
theme=list(np.unique(re.sub("[\(\[].*?[\)\]]", "",
    re.sub(","," ",
        re.sub(";"," ",' '.join(np.unique(projets['ThÃ©matiques'].values))))).split(' ')))
theme.remove('ET'),theme.remove('')

projets['theme']=[re.sub("[\(\[].*?[\)\]]", "",
    re.sub(","," ",
        re.sub(";"," ",' '.join(np.unique(projets['ThÃ©matiques'].values[i]))))).split(' ')
         for i in range(len(projets))]

<>:2: DeprecationWarning: invalid escape sequence \(
<>:7: DeprecationWarning: invalid escape sequence \(
<>:2: DeprecationWarning: invalid escape sequence \(
<>:7: DeprecationWarning: invalid escape sequence \(
<ipython-input-5-1a33340d6f12>:2: DeprecationWarning: invalid escape sequence \(
  theme=list(np.unique(re.sub("[\(\[].*?[\)\]]", "",
<ipython-input-5-1a33340d6f12>:7: DeprecationWarning: invalid escape sequence \(
  projets['theme']=[re.sub("[\(\[].*?[\)\]]", "",


On va concaténer le html des études avec leur thème puis nettoyer ce texte html pour l'avoir en texte txt

In [6]:
base=pd.concat([df_html,projets],axis=1)
base=base[base.texte.isna()==False]

from unidecode import unidecode
from bs4 import BeautifulSoup
import time
start=time.time()
base['clean']=[unidecode(re.sub(r'[^A-Za-z]',' ',
                BeautifulSoup(
                    base.texte.values[i],"html.parser").get_text()).lower())
                     for i in range(len(base))]
end=time.time()
print("Durée :",round((end-start)/60,2),"minutes")
base['clean']=[' '.join([i for i in base.clean.values[k].split() if len(i)>3]) for k in range(len(base))]

Durée : 13.38 minutes


In [7]:
base

,Unnamed: 0,num_etude,texte,num_etude,ThÃ©matiques,theme,clean
0,0.0,1003691.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",1112248,URBANISME ET CONSTRUCTION;TRANSPORTS,"[URBANISME, ET, CONSTRUCTION, TRANSPORTS]",munchhouse agence aquitaine safege avenue berl...
1,1.0,100689.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",1417225,Ã‰NERGIE,[Ã‰NERGIE],nier ovaed projet centrale photovolta solrecon...
2,2.0,100707.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",2528187,URBANISME ET CONSTRUCTION;ENVIRONNEMENT (dont ...,"[URBANISME, ET, CONSTRUCTION, ENVIRONNEMENT, ]",projet centrale photovolta reconversion carri ...
3,3.0,1007353.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",1493652,ENVIRONNEMENT (dont ICPE installation classÃ©e),"[ENVIRONNEMENT, ]",etude impact parc photovolta bernardan jouac j...
4,4.0,1007765.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",453463,ENVIRONNEMENT (dont ICPE installation classÃ©e),"[ENVIRONNEMENT, ]",sommaire ambule contexte table mati preambule ...
5,5.0,1008101.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",2561232,ENVIRONNEMENT (dont ICPE installation classÃ©e),"[ENVIRONNEMENT, ]",dossier demande autorisation environnementale ...
6,6.0,1012199.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",1533994,URBANISME ET CONSTRUCTION;ENVIRONNEMENT (dont ...,"[URBANISME, ET, CONSTRUCTION, ENVIRONNEMENT, ,...",claration fore fouesnan rapport finitif avril ...
7,7.0,1017145.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",2640983,URBANISME ET CONSTRUCTION,"[URBANISME, ET, CONSTRUCTION]",bordeaux tropole departement gironde commune m...
8,8.0,101872.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",2445064,ENVIRONNEMENT (dont ICPE installation classÃ©e),"[ENVIRONNEMENT, ]",artelia choletais audace pour ussir departemen...
9,9.0,102316.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",766331,Ã‰NERGIE,[Ã‰NERGIE],energie eolienne messac sarl projet eolien com...


##### 28 Avril 2021
On modifie ce code pour prendre en compte la structure du dictionnaire d'enjeu

In [8]:
def represent_word(word):
    import re
    from unidecode import unidecode
    # True-case, i.e. try to normalize sentence-initial capitals.
    # Only do this if the lower-cased form is more probable.
    text = word.replace("lire page","") #replce "lire page" par "" (en gros delete "lire page")
    text = unidecode(text) #take a unicode object and returns a string
    text=text.lower()
    text = re.sub(r'[^A-Za-z]',' ',str(text))
    #means any character that IS NOT a-z OR A-Z
    text = ' '.join([i for i in text.split() if len(i)>2])
    return text

In [36]:
thesaurus

,Enjeux,Dictionnaire
0,Biodiversité,"[biodiversité, milieu naturel, habitat naturel..."
1,Paysage et qualité de vie,"[paysage, cadre de vie, qualité de vie, intégr..."
2,Santé et sécurité des personnes,"[santé humaine, toxicité, bactériologique, qua..."
3,"Effets globaux (climat, énergie, ressources...)","[climat, changement climatique, réchauffement,..."
4,Préservation des sols,"[sols, artificialisation, consommation fonciér..."
5,Qualité de leau et ressource,"[eaux usées, eaux pluviales, eau de pluie, hyd..."
6,Déplacements,"[déplacements, transport, mobilité, véhicules,..."
7,Gestion des déchets,"[déchets, épandage, compostage, recyclage, mét..."


In [40]:
### A ne pas faire tourner quand on aura le vrai thesau (puisque les données ne proviendront pas d'un W2V)

thesau={}
for i in thesaurus.Enjeux:
    thesau[i]=[represent_word(k) for z in thesaurus[thesaurus.Enjeux==i].Dictionnaire for k in z ]
thesau

{'Biodiversité': ['biodiversite',
  'milieu naturel',
  'habitat naturel',
  'faune',
  'flore',
  'trame verte bleue tvb',
  'trame noire',
  'pollution lumineuse',
  'reservoir biologique',
  'continuite ecologique',
  'haie',
  'boisement',
  'prairie',
  'bocage',
  'milieu aquatique',
  'cours deau',
  'etat ecologique',
  'pollution',
  'debit reserve',
  'zone humide',
  'znieff',
  'natura',
  'espece protegee',
  'oiseaux',
  'chauve souris',
  'avifaune',
  'mammiferes',
  'poissons',
  'insectes',
  'herbier'],
 'Paysage et qualité de vie': ['paysage',
  'cadre vie',
  'qualite vie',
  'integration paysagere',
  'photomontage',
  'insertion visuelle',
  'point vue',
  'covisibilite',
  'monument historique',
  'site inscrit',
  'site classe',
  'nuisance',
  'commodite',
  'bruit',
  'vibrations',
  'acoustique',
  'emergence',
  '',
  'exposition',
  'odeur',
  'olfactif'],
 'Santé et sécurité des personnes': ['sante humaine',
  'toxicite',
  'bacteriologique',
  'qualite l

In [10]:
thesaurus #dictionnaire

,Enjeux,Dictionnaire
0,Biodiversité,"[biodiversité, milieu naturel, habitat naturel..."
1,Paysage et qualité de vie,"[paysage, cadre de vie, qualité de vie, intégr..."
2,Santé et sécurité des personnes,"[santé humaine, toxicité, bactériologique, qua..."
3,"Effets globaux (climat, énergie, ressources...)","[climat, changement climatique, réchauffement,..."
4,Préservation des sols,"[sols, artificialisation, consommation fonciér..."
5,Qualité de leau et ressource,"[eaux usées, eaux pluviales, eau de pluie, hyd..."
6,Déplacements,"[déplacements, transport, mobilité, véhicules,..."
7,Gestion des déchets,"[déchets, épandage, compostage, recyclage, mét..."


In [11]:
###### On définit E par l'ensemble des mots des enjeux, dont le proxy est le Thésaurus final fourni par la DREAL
###### Pour le moments nous n'avons ni thesaurus ni enjeux, en revanche nous avons LégiFrance
###### On définit E_T par le Thesaurus final de la Dreal et donc
###### On définit un ensemble E_T_ (prononcé E T prime) qui contient les mots du Thesaurus LégiFrance
import functools
import operator
#E_T_=functools.reduce(operator.iconcat, list(thesau.values()), [])
#E_T_=[unidecode(i.lower()) for i in E_T_]

###### On crée notre base de phrases pour le Word2Vec
import gensim
sentences =np.array([str(c).split() for c in base.clean.values])

###### On crée et entraine le modèle d'embedding
fenetre=15
minimum=1
d=300
e=10
satrt=time.time()
W2V=gensim.models.Word2Vec(size=d,window=fenetre,min_count=minimum)
W2V.build_vocab(sentences)
W2V.train(sentences,total_examples=W2V.corpus_count,epochs=e)
end=time.time()
print("Durée entraînement du Word2Vec :",round((end-start)/60,2),"minutes.")

/data/dataiku/data_dir_design/code-envs/python/THEO_/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/data/dataiku/data_dir_design/code-envs/python/THEO_/lib/python3.6/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


Durée entraînement du Word2Vec : 23.0 minutes.


In [12]:
base

,Unnamed: 0,num_etude,texte,num_etude,ThÃ©matiques,theme,clean
0,0.0,1003691.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",1112248,URBANISME ET CONSTRUCTION;TRANSPORTS,"[URBANISME, ET, CONSTRUCTION, TRANSPORTS]",munchhouse agence aquitaine safege avenue berl...
1,1.0,100689.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",1417225,Ã‰NERGIE,[Ã‰NERGIE],nier ovaed projet centrale photovolta solrecon...
2,2.0,100707.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",2528187,URBANISME ET CONSTRUCTION;ENVIRONNEMENT (dont ...,"[URBANISME, ET, CONSTRUCTION, ENVIRONNEMENT, ]",projet centrale photovolta reconversion carri ...
3,3.0,1007353.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",1493652,ENVIRONNEMENT (dont ICPE installation classÃ©e),"[ENVIRONNEMENT, ]",etude impact parc photovolta bernardan jouac j...
4,4.0,1007765.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",453463,ENVIRONNEMENT (dont ICPE installation classÃ©e),"[ENVIRONNEMENT, ]",sommaire ambule contexte table mati preambule ...
5,5.0,1008101.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",2561232,ENVIRONNEMENT (dont ICPE installation classÃ©e),"[ENVIRONNEMENT, ]",dossier demande autorisation environnementale ...
6,6.0,1012199.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",1533994,URBANISME ET CONSTRUCTION;ENVIRONNEMENT (dont ...,"[URBANISME, ET, CONSTRUCTION, ENVIRONNEMENT, ,...",claration fore fouesnan rapport finitif avril ...
7,7.0,1017145.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",2640983,URBANISME ET CONSTRUCTION,"[URBANISME, ET, CONSTRUCTION]",bordeaux tropole departement gironde commune m...
8,8.0,101872.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",2445064,ENVIRONNEMENT (dont ICPE installation classÃ©e),"[ENVIRONNEMENT, ]",artelia choletais audace pour ussir departemen...
9,9.0,102316.0,"﻿<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 T...",766331,Ã‰NERGIE,[Ã‰NERGIE],energie eolienne messac sarl projet eolien com...


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
#maxf=10000
vect = TfidfVectorizer(analyzer = "word", ngram_range=(1, 3), tokenizer = None, preprocessor = None,max_df=0.9,min_df=0.05)
train_features = vect.fit_transform(base.clean)

In [21]:
base_tfidf=pd.DataFrame(train_features.toarray(),columns=vect.get_feature_names())

In [22]:
vect.get_feature_names()

['aappma',
 'aasqa',
 'abaiss',
 'abaisse',
 'abaissement',
 'abaissement seuils',
 'abaisser',
 'abandon',
 'abandon activit',
 'abandon certaines',
 'abandon nich',
 'abandon pratiques',
 'abandon site',
 'abandon zone',
 'abandonn',
 'abandonn leur',
 'abandonn leur ouvrage',
 'abandonn ouvrage',
 'abandonn ouvrage incidences',
 'abandonn ouvrage projets',
 'abandonn pour',
 'abandonn susceptibles',
 'abandonn susceptibles engendrer',
 'abandonn titionnaire',
 'abandonn titionnaire ouvrage',
 'abandonner',
 'abandons',
 'abaque',
 'abattage',
 'abattage arbres',
 'abattages',
 'abattages arbres',
 'abattement',
 'abattoir',
 'abattoirs',
 'abattre',
 'abattu',
 'abattus',
 'abbatiale',
 'abbaye',
 'abbaye royale',
 'abbaye saint',
 'abeille',
 'abeilles',
 'abeilles bourdons',
 'abeilles bourdons fourmis',
 'abies',
 'abies aude',
 'abilis',
 'abilis dans',
 'abilis eaux',
 'abilis espaces',
 'abilis faible',
 'abilis pour',
 'abilis projet',
 'abilis sera',
 'abilis seront',
 'abil

On crée nos ensembles de mots

In [47]:
#Ensemble de mots du modèle
M=list(W2V.wv.vocab.keys())
len(M)

134257

In [49]:
M_=[m for m in vect.get_feature_names() if m in M]
print(len(M_))

17437


In [50]:
###### On définit E_T_tilde par l'intersection entre E_T_ et M
#Ensemble de mots du Thesaurus contenu dans le vocab du modèle
E_thesau={}
for i in thesau.keys():
    E_thesau[i]=[m for m in thesau[i] if m in M]

#E_T_tilde=[i for i in E_T_ if i in M]
#print(len(E_T_tilde))

##### On définit T comme l'ensemble des thèmes, donc T_tilde les :
# thèmes qui sont dans l'ensemble de mots
# On ne s'intéresse finalement plus au thème
#T_tilde=[i for i in [unidecode(z.lower()) for z in theme] if i in M]
#len(T_tilde)

#On va prendre les vecteurs de l'interseciton Thesaurus et Mots:
Vect_E_T_tilde={}
for i in thesau.keys():
    Vect_E_T_tilde[i]=[W2V[v] for v in E_thesau[i]]
#print(len(Vect_E_T_tilde))

#On récupère l'ensemble des vecteurs de chaque mot
Vect_M=[W2V[v] for v in M]
Vect_M_=[W2V[v] for v in M_]
print(len(Vect_M),len(Vect_M_))

#On récup les vecteurs des thèmes
#Vect_T_tilde=[W2V[v] for v in T_tilde]
#print(Vect_T_tilde)

/data/dataiku/data_dir_design/code-envs/python/THEO_/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/data/dataiku/data_dir_design/code-envs/python/THEO_/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/data/dataiku/data_dir_design/code-envs/python/THEO_/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


134257 17437


In [52]:
#Vect_E_T_tilde
#Vect_M_

{'Biodiversité': [array([ 7.66292214e-02,  9.25372448e-03, -9.70921814e-02, -4.34742756e-02,
          1.10475875e-01, -1.21788464e-01, -1.73415672e-02, -2.07972273e-01,
          2.28459403e-01, -1.26912296e-01,  1.20256804e-01,  1.36423074e-02,
         -4.93758470e-02, -4.26257625e-02,  1.74214199e-01,  1.54375300e-01,
          5.95302507e-02,  3.26370262e-02,  6.61920086e-02,  1.85997352e-01,
          4.19144705e-02, -1.16241515e-01,  1.07186191e-01, -1.35429889e-01,
         -1.95734203e-02, -1.02775626e-01,  1.63730323e-01, -3.42693329e-01,
          3.41116846e-01,  3.13717008e-01,  2.11417735e-01,  9.43039581e-02,
          8.88167396e-02, -2.79315203e-01, -6.32146597e-02, -5.73798195e-02,
         -1.22005679e-01, -1.29718548e-02, -1.92759037e-01, -3.76545899e-02,
          2.06390768e-01,  2.85085529e-01,  3.33967358e-02,  2.32270196e-01,
          5.88780269e-02, -3.64796311e-01, -2.48043358e-01,  1.98123679e-01,
         -7.99620226e-02, -9.87299159e-02, -3.32346372e-02, 

In [44]:
def euclid(x):
    import numpy as np
    d=np.sqrt(sum([i**2 for i in x]))
    return d

def cos_sim(x,y):
    a=x@y
    l=euclid(x)*euclid(y)
    sim=a/l
    return sim

### A voir pour changer la fonction de cosim, là tu utilises la mienne, mais elle est vraiment pas optimiser
Tu peux utiliser ça plutôt, qui vient directement de torch :

In [81]:
#!pip install torch
import torch
cosim=torch.nn.CosineSimilarity(dim=0)
x=torch.rand(10)
y=torch.rand(10)
print(x,y)
cosim(x,y)

tensor([0.1590, 0.2606, 0.6567, 0.8023, 0.9729, 0.3449, 0.8691, 0.5024, 0.5821,
        0.8360]) tensor([0.8006, 0.4516, 0.5958, 0.5859, 0.1290, 0.6401, 0.9073, 0.8014, 0.3378,
        0.0431])


tensor(0.7379)

In [0]:
from joblib import Parallel,delayed
import time
duree=[]
absi=[]
import psutil
cpu=psutil.cpu_count()
for i in range(1,cpu):
    absi.append(i)
    start=time.time()
    Parallel(n_jobs=i,verbose=0)(delayed(cos_sim)(Vect_M[0],v) for v in Vect_E_T_tilde['construction'])
    end=time.time()
    duree.append(end-start)

import matplotlib.pyplot as plt
f,a=plt.subplots(1,figsize=(12,6))
a.plot(absi,duree)
a.set(xlabel="Nombre de job",ylabel='Durée',
      title='Durée en fonction du degré de parallélisation')

In [0]:
jobs=[duree.index(i) for i in sorted(duree)[0:1]][0]+1

In [61]:
from joblib import Parallel,delayed
import time
from tqdm import tqdm
import psutil
#start=time.time()
#cos_moyen=[np.mean(
#    Parallel(n_jobs=jobs,verbose=0)(delayed(cos_sim)(m,v) for v in Vect_E_T_tilde))
#  for m in tqdm(Vect_M)]
#end=time.time()
#print("La parallélisation a durée :",round((end-start)/60,3),"minutes")


###########Par thème

def cos_moyen(Vect_E_T_tilde,Vect_M,jobs,verb=0):
    from joblib import Parallel,delayed
    import numpy as np
    from tqdm import tqdm
    cos=[np.mean(
        Parallel(n_jobs=jobs,verbose=verb)(delayed(cos_sim)(m,v) for v in Vect_E_T_tilde))
      for m in tqdm(Vect_M)]
    return cos

cpu=psutil.cpu_count()
print(cpu,"CPU vont être utilisés")

from functools import partial

cos_moyen_=partial(cos_moyen,Vect_M=Vect_M,jobs=cpu,verb=0)
 #cos_thesau_={}

start=time.time()
cos_thesau_=Parallel(n_jobs=cpu)(delayed(cos_moyen_)(Vect_E_T_tilde[i]) for i in tqdm(Vect_E_T_tilde.keys()))
#[np.mean(Parallel(n_jobs=jobs,verbose=0)(delayed(cos_sim)(m,v) for v in Vect_E_T_tilde[i])) for m in tqdm(Vect_M)]
end=time.time()
print("La parallélisation a durée :",round((end-start)/60,3),"minutes")

# pd.DataFrame(Vect_M,columns=M)

100%|██████████| 8/8 [00:00<00:00, 1344.38it/s]

64 CPU vont être utilisés


La parallélisation a durée : 586.249 minutes


In [62]:
import pickle
da=dataiku.Folder('XXZ13n5V')
pickle.dump(cos_thesau_,open(da.get_path()+"/cos_thesau.pickle",'wb'))

In [56]:
cos_thesau_

[[0.141950246395344,
  0.03443845122017098,
  0.0023328646181882493,
  0.16549823078066528,
  -0.09232928649444219,
  -0.15529456615790652,
  0.041740367927670774,
  0.06134917915605601,
  -0.019015561907175536,
  0.2496707156965234,
  -0.11580475923941172,
  -0.049012716712373804,
  0.07973358524102384,
  -0.0520082231533746,
  -0.14235349008246995,
  0.0016909580189495843,
  0.11638980320804904,
  -0.0009467391293187938,
  0.12040933101462054,
  -0.10007075024292977,
  0.07530399143794772,
  0.30365548103303164,
  0.25893708128172715,
  0.36610571155498384,
  0.05348258947302355,
  -0.0756093340224531,
  0.0559768202862025,
  0.10910182335240065,
  -0.02008584210588685,
  0.17906392930338155,
  0.21731930622656498,
  0.07016813665564281,
  0.0420372493869764,
  -0.054163603562746646,
  0.015704694428537096,
  0.2940929599562867,
  0.012077290134561339,
  0.20412747069770534,
  0.2807867516111062,
  0.22633861613979,
  0.1897310747317863,
  0.3625717713879221,
  -0.12197580789055996,


In [0]:
import pickle
da=dataiku.Folder('XXZ13n5V')
cos_thesau=pickle.load(open(da.get_path()+"/cos_thesau.pickle",'rb')

In [0]:
len(cos_thesau),len(cos_thesau[0])

In [60]:
M_

['aappma',
 'aasqa',
 'abaiss',
 'abaisse',
 'abaissement',
 'abaisser',
 'abandon',
 'abandonn',
 'abandonner',
 'abandons',
 'abaque',
 'abattage',
 'abattages',
 'abattement',
 'abattoir',
 'abattoirs',
 'abattre',
 'abattu',
 'abattus',
 'abbatiale',
 'abbaye',
 'abeille',
 'abeilles',
 'abies',
 'abilis',
 'abilisant',
 'abilisation',
 'abiliser',
 'abilit',
 'abiotique',
 'abiotiques',
 'able',
 'ables',
 'aboiements',
 'abondamment',
 'abondance',
 'abondances',
 'abondant',
 'abondante',
 'abondantes',
 'abondants',
 'abonde',
 'abonn',
 'abonnement',
 'abordable',
 'aborde',
 'abordent',
 'aborder',
 'abouti',
 'aboutir',
 'aboutira',
 'aboutissant',
 'aboutissement',
 'aboutissent',
 'aboutit',
 'aboyeur',
 'abrasion',
 'abreuvage',
 'abreuvement',
 'abreuver',
 'abreuvoir',
 'abreuvoirs',
 'abri',
 'abris',
 'abrit',
 'abritait',
 'abritant',
 'abrite',
 'abritent',
 'abriter',
 'abritera',
 'abrog',
 'abrogeant',
 'abrupte',
 'abruptes',
 'abrupts',
 'absences',
 'absent',


In [63]:
N=10
mots_thesau={}
for cos_m in cos_thesau_:
    i=pd.Series(cos_m).nlargest(N)
    mots_thesau[list(thesau.keys())[cos_thesau_.index(cos_m)]]=[M[k] for k in i.index.values.tolist()]

mots_thesau

{'Biodiversité': ['cili',
  'insectes',
  'entomofaune',
  'robinier',
  'lique',
  'asconit',
  'insecte',
  'tarier',
  'rollier',
  'germandr'],
 'Paysage et qualité de vie': ['commodit',
  'bruits',
  'vibrations',
  'acoustique',
  'mesurant',
  'ambiant',
  'sonore',
  'sonores',
  'lumineuses',
  'laeq'],
 'Santé et sécurité des personnes': ['explosion',
  'risque',
  'torrentielles',
  'radioactif',
  'avalanches',
  'radon',
  'accident',
  'submersion',
  'incendies',
  'isme'],
 'Effets globaux (climat, énergie, ressources...)': ['rechauffement',
  'climat',
  'climatique',
  'changement',
  'franciliens',
  'tiser',
  'energetique',
  'procure',
  'silience',
  'adaptation'],
 'Préservation des sols': ['artificialisation',
  'fertiles',
  'sols',
  'urbanisation',
  'mixit',
  'dominante',
  'fortement',
  'vocation',
  'enlevage',
  'occupation'],
 'Qualité de leau et ressource': ['semestrielle',
  'trimestrielle',
  'riologique',
  'effluent',
  'chlore',
  'chlorures',
 

In [71]:
da=dataiku.Folder('XXZ13n5V')
pickle.dump(mots_thesau,open(da.get_path()+"/mots_thesau_W2V.pickle",'wb'))

In [68]:
mots_thesau_2={}
for i in thesau.keys():
    ouais=[]
    for k in mots_thesau[i]:
        if k not in thesau[i]:
            ouais.append(k)
    mots_thesau_2[i]=ouais
mots_thesau_2

{'Biodiversité': ['cili',
  'entomofaune',
  'robinier',
  'lique',
  'asconit',
  'insecte',
  'tarier',
  'rollier',
  'germandr'],
 'Paysage et qualité de vie': ['commodit',
  'bruits',
  'mesurant',
  'ambiant',
  'sonore',
  'sonores',
  'lumineuses',
  'laeq'],
 'Santé et sécurité des personnes': ['risque',
  'torrentielles',
  'radioactif',
  'avalanches',
  'incendies',
  'isme'],
 'Effets globaux (climat, énergie, ressources...)': ['climatique',
  'changement',
  'franciliens',
  'tiser',
  'energetique',
  'procure',
  'silience'],
 'Préservation des sols': ['fertiles',
  'mixit',
  'dominante',
  'fortement',
  'vocation',
  'enlevage',
  'occupation'],
 'Qualité de leau et ressource': ['semestrielle',
  'trimestrielle',
  'riologique',
  'effluent',
  'chlore',
  'chlorures',
  'hydrocarbures',
  'doseuse',
  'phosphat',
  'phosphates'],
 'Déplacements': ['ferroviaire',
  'carrefour',
  'rafic',
  'barreau',
  'desservant',
  'desserte',
  'gobain',
  'automobilistes',
  't

In [70]:
for i in mots_thesau_2.keys():
    print("################### ",i)
    print(mots_thesau_2[i],"\n")

###################  Biodiversité
['cili', 'entomofaune', 'robinier', 'lique', 'asconit', 'insecte', 'tarier', 'rollier', 'germandr'] 

###################  Paysage et qualité de vie
['commodit', 'bruits', 'mesurant', 'ambiant', 'sonore', 'sonores', 'lumineuses', 'laeq'] 

###################  Santé et sécurité des personnes
['risque', 'torrentielles', 'radioactif', 'avalanches', 'incendies', 'isme'] 

###################  Effets globaux (climat, énergie, ressources...)
['climatique', 'changement', 'franciliens', 'tiser', 'energetique', 'procure', 'silience'] 

###################  Préservation des sols
['fertiles', 'mixit', 'dominante', 'fortement', 'vocation', 'enlevage', 'occupation'] 

###################  Qualité de leau et ressource
['semestrielle', 'trimestrielle', 'riologique', 'effluent', 'chlore', 'chlorures', 'hydrocarbures', 'doseuse', 'phosphat', 'phosphates'] 

###################  Déplacements
['ferroviaire', 'carrefour', 'rafic', 'barreau', 'desservant', 'desserte', 'go

In [0]:
cosmax=max(cos_moyen)
index_max=cos_moyen.index(cosmax)

In [0]:
N=10
i = pd.Series(cos_thesau[0]).nlargest(N)
i.index.values.tolist()

In [0]:
pd.DataFrame(cos_moyen,M)

In [0]:
len(vocab_theme)

In [0]:
thesaurus_enjeu_df = pd.DataFrame(cos_moyen) # Compute a Pandas dataframe to write into Thesaurus_enjeu


# Write recipe outputs
thesaurus_enjeu = dataiku.Dataset("Thesaurus_enjeu")
thesaurus_enjeu.write_with_schema(thesaurus_enjeu_df)